## Three soldiers trading

In [4]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
from ps_candlestick import get_type_candlestick
from ps_candlestick import has_reversal_pattern

In [5]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [6]:
data = dataset.copy()
data["current"] = data.index + pd.DateOffset(minutes=5)
data['1st_body'] = data['Close'] - data['Open']
data['2nd_body'] = data['1st_body'].shift(1)
data['3rd_body'] = data['1st_body'].shift(2)
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma'] = data['above_ma'].rolling(150).sum()
data['total_below_ma'] = data['below_ma'].rolling(150).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
data.dropna(inplace=True)

## Strategy

In [7]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at x0 pips
    sl_step = 2
    # Takeprofit at x0 pips(R/R = 1/3)
    tp_step = 6
    for i, row in prepared_data.iterrows():
        if 100*row.name.hour + row.name.minute < 1420:
            if has_open_deal:
                if type_open_deal == 'short':
                    # Stoploss
                    if row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['Low'] < entry_price - tp_step:
                            profit = tp_step
                            exit_price = entry_price - tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                            has_open_deal = False
                            type_open_deal = ''
                elif type_open_deal == 'long':
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['High'] > entry_price + tp_step:
                            profit = tp_step
                            exit_price = entry_price + tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                            has_open_deal = False
                            type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = (row['Close'] + row['Open'])/2
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = (row['Close'] + row['Open'])/2
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
        else:
            if 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
    return _trades

In [8]:
def cal_signal_v1(row):
    signal = ''
    if (row['trend'] == 'up' or row['trend'] == 'switch') and row['1st_body'] > 0 and row['2nd_body'] > 0 and row['3rd_body'] > 0:
        signal = 'long'
    elif (row['trend'] == 'down' or row['trend'] == 'switch') and row['1st_body'] < 0 and row['2nd_body'] < 0 and row['3rd_body'] < 0:
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_v1(r), axis=1)

In [9]:
trade_result = trade_simulation(data)

In [10]:
total_trades = len(trade_result)
profit = trade_result.Profit.sum()
trading_fee = total_trades * 0.3
win_rate = len(trade_result[trade_result.Profit > 0]) / total_trades
display_data = {
    'trades' : total_trades,
    'profit' : profit,
    'win rate' : win_rate,
    'trading_fee' : trading_fee,
    'profit_after_fee' : profit - trading_fee
}
display_data

{'trades': 3239,
 'profit': 3475.45,
 'win rate': 0.4195739425748688,
 'trading_fee': 971.6999999999999,
 'profit_after_fee': 2503.75}

In [11]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-16 10:35:00,941.70,2018-08-16 10:45:00,939.70,Long,-2.00
1,2018-08-16 11:25:00,938.05,2018-08-16 14:05:00,944.05,Long,6.00
2,2018-08-17 10:35:00,950.80,2018-08-17 11:15:00,952.80,Short,-2.00
3,2018-08-17 11:20:00,953.15,2018-08-17 13:30:00,951.15,Long,-2.00
4,2018-08-17 13:55:00,947.00,2018-08-17 14:05:00,949.00,Short,-2.00
...,...,...,...,...,...,...
3234,2024-07-08 11:15:00,1307.25,2024-07-08 14:25:00,1312.00,Long,4.75
3235,2024-07-09 09:40:00,1307.20,2024-07-09 10:30:00,1309.20,Short,-2.00
3236,2024-07-09 13:05:00,1318.10,2024-07-09 13:35:00,1316.10,Long,-2.00
3237,2024-07-10 10:05:00,1319.50,2024-07-10 10:50:00,1317.50,Long,-2.00


In [12]:
data[data.index == '2024-07-10 10:05:00']

,Open,High,Low,Close,Volume,current,1st_body,2nd_body,3rd_body,ema_f,ema_f_shift,ema_l,ema_l_shift,ma_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,signal
Date,,,,,,,,,,,,,,,,,,,,
2024-07-10 10:05:00,1318.9,1321.2,1318.9,1320.1,6719,2024-07-10 10:10:00,1.2,0.8,1.2,1318.72051,1318.030765,1318.047717,1317.831688,1318.72,1,0,80.0,70.0,up,long


In [13]:
data[data.index == '2024-07-10 11:25:00']

,Open,High,Low,Close,Volume,current,1st_body,2nd_body,3rd_body,ema_f,ema_f_shift,ema_l,ema_l_shift,ma_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,signal
Date,,,,,,,,,,,,,,,,,,,,
2024-07-10 11:25:00,1319.2,1320.5,1319.1,1319.5,4160,2024-07-10 11:30:00,0.3,0.1,0.7,1318.993438,1318.740158,1318.704249,1318.620486,1318.92,1,0,86.0,64.0,up,long


In [14]:
data[data.index == '2024-07-08 11:15:00']

,Open,High,Low,Close,Volume,current,1st_body,2nd_body,3rd_body,ema_f,ema_f_shift,ema_l,ema_l_shift,ma_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,signal
Date,,,,,,,,,,,,,,,,,,,,
2024-07-08 11:15:00,1307.2,1307.4,1306.8,1307.3,2443,2024-07-08 11:20:00,0.1,1.0,0.4,1307.732376,1307.948564,1310.495011,1310.831328,1311.2,0,1,88.0,62.0,up,long
